<a href="https://colab.research.google.com/github/nananair/Research-NLP-projects/blob/main/Distributional_analysis_toolkit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook accompanies the article *"Methodological pathways to studying the shareability of news values"*.  

It shows how to perform a quartile-based distributional analysis to explore the shareability of news values.  

The notebook is designed for accessibility: simply upload your labelled dataset (Excel or CSV) in Google Colab, and the analysis will run automatically. No coding experience is required.

⚠️ Make sure your labelled dataset includes a column for each news value, even if some values were not identified in your corpus (e.g., an empty or zero-filled column for "Negativity").

⚠️ If your dataset contains multiple case studies (e.g., different countries), make sure it includes a column named "Dataset" that labels each row with its case. If your dataset only covers a single case, you do not need to add this column—the notebook will automatically create a default label (“All articles”) so the analysis still works.

⚠️ Your dataset must include at least one column with share counts.  
- If you analyse a single platform, name the column **"Share_counts"**.  
- If you analyse multiple platforms, name them **"Share_counts_Twitter"** and/or **"Share_counts_Facebook"**.  
- The code will automatically detect which columns are present and run the analysis separately (and combined, if both are available).  




In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm


In [2]:
from google.colab import files

print("Please upload your labelled dataset (Excel or CSV).")
uploaded = files.upload()

for filename in uploaded.keys():
    if filename.endswith(".xlsx"):
        df = pd.read_excel(filename)
    elif filename.endswith(".csv"):
        df = pd.read_csv(filename)
    else:
        raise ValueError("File must be .xlsx or .csv")
    break

print("Dataset loaded successfully")
df.head()


Please upload your labelled dataset (Excel or CSV).


Saving updated_thesis_dataset_v2.xlsx to updated_thesis_dataset_v2 (3).xlsx
Dataset loaded successfully


,Articles,Outlet_type,Word_length,Share_counts_Twitter,Share_counts_Facebook,Dataset,Eliteness,Personalisation,Superlativeness,Negativity,Timeliness,Unexpectedness,Impact,Proximity,Positivity,Consonance
0,Second police officer died by suicide followin...,Online-only,247.0,21800,21200,USA,4,0,5,6,1,0,0,8,0,NaN
1,Rioters breached US Capitol security on Wednes...,Popular,622.0,19800,24100,USA,11,4,8,9,10,0,2,11,0,NaN
2,‘They Got a Officer!’: How a Mob Dragged and B...,Quality,441.0,12900,23400,USA,2,0,9,15,0,0,0,9,0,NaN
3,"As the D.C. police clear the Capitol grounds, ...",Quality,468.0,16400,20100,USA,7,0,2,4,7,0,2,13,0,NaN
4,Now it’s sinking in: Wednesday’s Capitol Hill ...,Popular,1042.0,17200,23700,USA,9,9,23,10,8,11,2,11,0,NaN


In [3]:
df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Articles               192 non-null    object 
 1   Outlet_type            192 non-null    object 
 2   Word_length            192 non-null    float64
 3   Share_counts_Twitter   192 non-null    int64  
 4   Share_counts_Facebook  192 non-null    int64  
 5   Dataset                192 non-null    object 
 6   Eliteness              192 non-null    int64  
 7   Personalisation        192 non-null    int64  
 8   Superlativeness        192 non-null    int64  
 9   Negativity             192 non-null    int64  
 10  Timeliness             192 non-null    int64  
 11  Unexpectedness         192 non-null    int64  
 12  Impact                 192 non-null    int64  
 13  Proximity              192 non-null    int64  
 14  Positivity             192 non-null    int64  
 15  Conson

,Articles,Outlet_type,Word_length,Share_counts_Twitter,Share_counts_Facebook,Dataset,Eliteness,Personalisation,Superlativeness,Negativity,Timeliness,Unexpectedness,Impact,Proximity,Positivity,Consonance
0,Second police officer died by suicide followin...,Online-only,247.0,21800,21200,USA,4,0,5,6,1,0,0,8,0,NaN
1,Rioters breached US Capitol security on Wednes...,Popular,622.0,19800,24100,USA,11,4,8,9,10,0,2,11,0,NaN
2,‘They Got a Officer!’: How a Mob Dragged and B...,Quality,441.0,12900,23400,USA,2,0,9,15,0,0,0,9,0,NaN
3,"As the D.C. police clear the Capitol grounds, ...",Quality,468.0,16400,20100,USA,7,0,2,4,7,0,2,13,0,NaN
4,Now it’s sinking in: Wednesday’s Capitol Hill ...,Popular,1042.0,17200,23700,USA,9,9,23,10,8,11,2,11,0,NaN


In [4]:
# Case/Grouping setup: handle 1 case or multiple cases
CASE_COL = "Dataset"

if CASE_COL not in df.columns:
    df[CASE_COL] = "All articles"


In [7]:
news_values = ["Eliteness","Personalisation","Superlativeness","Negativity",
               "Timeliness","Unexpectedness","Impact","Proximity","Positivity"]

for nv in news_values:
    if nv not in df.columns:
        df[nv] = 0
df[news_values] = df[news_values].apply(pd.to_numeric, errors="coerce").fillna(0)

CASE_COL = "Dataset"
if CASE_COL not in df.columns:
    df[CASE_COL] = "All articles"

share_cols = []
if "Share_counts" in df.columns: share_cols.append("Share_counts")
if "Share_counts_Twitter" in df.columns: share_cols.append("Share_counts_Twitter")
if "Share_counts_Facebook" in df.columns: share_cols.append("Share_counts_Facebook")
make_combined = set(["Share_counts_Twitter","Share_counts_Facebook"]).issubset(df.columns)

QUART_LABELS = ["Q1","Q2","Q3","Q4"]
quartile_cat = pd.CategoricalDtype(categories=QUART_LABELS, ordered=True)

def build_table_for_metric(data, metric_name, series, quartiles_within_case=True):
    """
    Returns a DataFrame: Case Study | Share Metric | Share_Quartile | mean NVs...
    - Always targets quartiles; uses rank fallback if qcut fails (ties/small N).
    - Empty bins later get 0 (so no NaNs in the output).
    """
    tmp = data.copy()
    tmp["_shares_"] = pd.to_numeric(series, errors="coerce")
    tmp = tmp[tmp["_shares_"].notna()].copy()

    if quartiles_within_case:
        parts = []
        for case, g in tmp.groupby(CASE_COL, dropna=False):
            try:
                g["Share_Quartile"] = pd.qcut(g["_shares_"], q=4, labels=QUART_LABELS)
            except ValueError:
                r = g["_shares_"].rank(method="average")
                g["Share_Quartile"] = pd.qcut(r, q=4, labels=QUART_LABELS)
            parts.append(g)
        tmp = pd.concat(parts, ignore_index=True)
    else:
        try:
            tmp["Share_Quartile"] = pd.qcut(tmp["_shares_"], q=4, labels=QUART_LABELS)
        except ValueError:
            r = tmp["_shares_"].rank(method="average")
            tmp["Share_Quartile"] = pd.qcut(r, q=4, labels=QUART_LABELS)

    if tmp["Share_Quartile"].dtype.name != "category":
        tmp["Share_Quartile"] = tmp["Share_Quartile"].astype(quartile_cat)

    grouped = (
        tmp.groupby([CASE_COL, "Share_Quartile"], observed=False)[news_values]
           .mean()
           .reset_index()
           .rename(columns={CASE_COL: "Case Study"})
    )

    filled = []
    for case, g in grouped.groupby("Case Study", dropna=False):
        idx = pd.MultiIndex.from_product([[case], QUART_LABELS], names=["Case Study","Share_Quartile"])
        g = g.set_index(["Case Study","Share_Quartile"]).reindex(idx).reset_index()
        g[news_values] = g[news_values].fillna(0)
        filled.append(g)

    grouped = pd.concat(filled, ignore_index=True)
    grouped.insert(1, "Share Metric", metric_name)
    return grouped

def add_counts_and_clean(tbl, news_values, quart_labels=QUART_LABELS):
    """
    Adds n_articles per Case×Metric×Quartile, sorts rows, rounds NV means.
    """
    out = tbl.copy()
    counts_list = []
    for (metric, case) in out[["Share Metric","Case Study"]].drop_duplicates().itertuples(index=False):
        d = df[df[CASE_COL] == case].copy()
        if metric in d.columns:
            shares = pd.to_numeric(d[metric], errors="coerce")
        elif metric == "Share_counts_Combined":
            shares = pd.to_numeric(d.get("Share_counts_Twitter", 0), errors="coerce") + \
                     pd.to_numeric(d.get("Share_counts_Facebook", 0), errors="coerce")
        else:
            continue
        d = d.assign(_shares_=shares).dropna(subset=["_shares_"])
        try:
            q = pd.qcut(d["_shares_"], q=4, labels=quart_labels)
        except ValueError:
            r = d["_shares_"].rank(method="average")
            q = pd.qcut(r, q=4, labels=quart_labels)
        cnt = (pd.DataFrame({"Case Study":[case]*len(q),
                             "Share Metric":[metric]*len(q),
                             "Share_Quartile":q})
               .groupby(["Case Study","Share Metric","Share_Quartile"])
               .size().reset_index(name="n_articles"))
        counts_list.append(cnt)

    counts = (pd.concat(counts_list, ignore_index=True)
              if counts_list else
              pd.DataFrame(columns=["Case Study","Share Metric","Share_Quartile","n_articles"]))

    out = out.merge(counts, on=["Case Study","Share Metric","Share_Quartile"], how="left")
    out["Share_Quartile"] = pd.Categorical(out["Share_Quartile"], categories=quart_labels, ordered=True)
    out = out.sort_values(["Case Study","Share Metric","Share_Quartile"]).reset_index(drop=True)
    out[news_values] = out[news_values].round(3)
    out["n_articles"] = out["n_articles"].fillna(0).astype(int)
    return out

all_tables, excel_sheets = [], {}

for col in share_cols:
    tbl = build_table_for_metric(df, col, df[col], quartiles_within_case=True)
    all_tables.append(tbl)
    excel_sheets[col] = tbl

if make_combined:
    df["Share_counts_Combined"] = df["Share_counts_Twitter"] + df["Share_counts_Facebook"]
    tblc = build_table_for_metric(df, "Share_counts_Combined", df["Share_counts_Combined"], quartiles_within_case=True)
    all_tables.append(tblc)
    excel_sheets["Share_counts_Combined"] = tblc

final_table = pd.concat(all_tables, ignore_index=True)

clean_table = add_counts_and_clean(final_table, news_values)

pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_rows", 100)

for metric in clean_table["Share Metric"].unique():
    print(f"\n================  {metric}  ================")
    subm = clean_table[clean_table["Share Metric"] == metric]
    for case in subm["Case Study"].unique():
        print(f"\n--- Case Study: {case} ---")
        case_tbl = subm[subm["Case Study"] == case][
            ["Case Study","Share Metric","Share_Quartile","n_articles"] + news_values
        ]
        display(case_tbl)

        report = case_tbl.set_index("Share_Quartile")[["n_articles"] + news_values]
        display(report)




/tmp/ipython-input-2495236466.py:106: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["Case Study","Share Metric","Share_Quartile"])
/tmp/ipython-input-2495236466.py:106: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["Case Study","Share Metric","Share_Quartile"])
/tmp/ipython-input-2495236466.py:106: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["Case Study","Share Metric","Share_Quartile"])
/


================  Share_counts_Combined  ================

--- Case Study: Benin ---


/tmp/ipython-input-2495236466.py:106: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["Case Study","Share Metric","Share_Quartile"])
/tmp/ipython-input-2495236466.py:106: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["Case Study","Share Metric","Share_Quartile"])
/tmp/ipython-input-2495236466.py:106: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["Case Study","Share Metric","Share_Quartile"])


,Case Study,Share Metric,Share_Quartile,n_articles,Eliteness,Personalisation,Superlativeness,Negativity,Timeliness,Unexpectedness,Impact,Proximity,Positivity
0,Benin,Share_counts_Combined,Q1,5,14.2,0.8,0.8,6.0,3.0,0.0,0.0,7.2,0.0
1,Benin,Share_counts_Combined,Q2,5,12.8,0.6,1.2,7.4,3.4,0.0,0.0,6.2,0.0
2,Benin,Share_counts_Combined,Q3,5,9.8,3.0,1.4,6.4,3.0,0.0,0.0,9.2,0.0
3,Benin,Share_counts_Combined,Q4,5,15.0,0.6,2.6,8.8,4.6,0.0,0.0,9.8,0.0


,n_articles,Eliteness,Personalisation,Superlativeness,Negativity,Timeliness,Unexpectedness,Impact,Proximity,Positivity
Share_Quartile,,,,,,,,,,
Q1,5,14.2,0.8,0.8,6.0,3.0,0.0,0.0,7.2,0.0
Q2,5,12.8,0.6,1.2,7.4,3.4,0.0,0.0,6.2,0.0
Q3,5,9.8,3.0,1.4,6.4,3.0,0.0,0.0,9.2,0.0
Q4,5,15.0,0.6,2.6,8.8,4.6,0.0,0.0,9.8,0.0


...

,n_articles,Eliteness,Personalisation,Superlativeness,Negativity,Timeliness,Unexpectedness,Impact,Proximity,Positivity
Share_Quartile,,,,,,,,,,
Q1,5,22.000,0.600,2.000,4.200,5.600,0.0,3.000,5.800,1.800
Q2,3,19.667,1.667,1.667,3.333,3.000,0.0,2.000,6.333,3.000
Q3,4,27.250,1.750,1.250,2.750,5.500,0.0,3.750,7.750,2.250
Q4,3,32.000,3.000,3.333,5.000,7.333,0.0,7.667,8.333,3.333



--- Case Study: USA ---


,Case Study,Share Metric,Share_Quartile,n_articles,Eliteness,Personalisation,Superlativeness,Negativity,Timeliness,Unexpectedness,Impact,Proximity,Positivity
80,USA,Share_counts_Twitter,Q1,5,3.0,2.20,6.20,13.8,2.80,0.40,2.80,11.8,0.0
81,USA,Share_counts_Twitter,Q2,5,13.0,4.00,10.80,9.6,7.00,2.60,1.80,13.2,0.0
82,USA,Share_counts_Twitter,Q3,4,30.0,8.75,9.25,11.5,2.75,2.25,3.75,14.5,0.0
83,USA,Share_counts_Twitter,Q4,5,54.6,20.00,23.00,27.6,15.60,3.00,13.80,35.8,0.0


,n_articles,Eliteness,Personalisation,Superlativeness,Negativity,Timeliness,Unexpectedness,Impact,Proximity,Positivity
Share_Quartile,,,,,,,,,,
Q1,5,3.0,2.20,6.20,13.8,2.80,0.40,2.80,11.8,0.0
Q2,5,13.0,4.00,10.80,9.6,7.00,2.60,1.80,13.2,0.0
Q3,4,30.0,8.75,9.25,11.5,2.75,2.25,3.75,14.5,0.0
Q4,5,54.6,20.00,23.00,27.6,15.60,3.00,13.80,35.8,0.0


Exported 'case_by_case_quartile_tables.xlsx' with a sheet per (metric × case).


In [9]:
with pd.ExcelWriter("shareworthiness_results.xlsx") as writer:
    clean_table.to_excel(writer, sheet_name="ALL_long", index=False)

    for (metric, case), g in clean_table.groupby(["Share Metric","Case Study"], sort=False):
        sheet = f"{metric}_{case}"[:31]  # Excel sheet name limit
        g[["Case Study","Share Metric","Share_Quartile","n_articles"] + news_values] \
          .to_excel(writer, sheet_name=sheet, index=False)

from google.colab import files
files.download("shareworthiness_results.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Done!  

You have successfully run a shareworthiness analysis on your dataset.  
This notebook is fully reproducible and can be adapted to any dataset annotated with news values.  

